In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys
# print(sys.executable) #check jupyter kernel
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from data.simu_setting import prox_data
from src.proxITR import proxITR
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler as Scaler
from sklearn.kernel_approximation import Nystroem

### Value Function

In [3]:
def value_fun(d,A,Y,X,U):
    A = A.to_numpy().reshape(-1)
    Y = Y.to_numpy().reshape(-1)
    U = U.to_numpy().reshape(-1)
    X = X.to_numpy()
    return np.mean((np.abs((np.sign(A-0.5)-d))<0.5)*Y*(1. + np.exp(np.sign(A-0.5)*(0.09375+X @ [0.1875, 0.1875] -0.25*U))))

# Examples for Different Simulation Settings

## Scenario L1

In [4]:
# Setting
linearity    = 'linear'
Xonly        = 'XW'
# Selected quantile for dPESS
qtl = 0.4

In [5]:
# Generate Test Data
test = prox_data(500000, add_noise=False).gen_Y(linearity=linearity, Xonly = Xonly)
value_fun(test[["GOR"]].to_numpy(int).reshape(-1), test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])

6.2625704480648805

In [6]:
# Generate Training Data
samp_size = 2000
train = prox_data(samp_size=samp_size, add_noise=True).gen_Y(linearity=linearity, Xonly = Xonly)

### With 8 noisy variables in $L$

In [7]:
# Learn ITRs
rhos = np.power(2.,-np.arange(-2,7))
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
                                learning_rate= 0.1, n_epoch=2000, opt='LBFGS', verbose=True, qtl = qtl)
d1_XZ = proxy.fit_d1_XZ_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_XZ = d1_XZ(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","Z"]])

d2_XW = proxy.fit_d2_XW_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_XW = d2_XW(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","W"]])

d_DR  = proxy.fit_d_DR_cv(gamma_f='auto', n_gamma_hs=10, 
        h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos, learning_rate=1e-2)
p_d_DR  = d_DR(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

d1_X  = proxy.fit_d1_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_X  = d1_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

d2_X  = proxy.fit_d2_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_X  = d2_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

d1_XZ
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.9514875 5.9514875 5.9514875 6.0924215 6.28064   6.312313  6.3245764
 6.32756   6.331319 ]
rho_best: 0.015625
qtl_best: 6.264737510681153
d2_XW
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.8791139  5.8791139  5.8791139  5.8791139  5.8791139  5.91107977
 6.03214941 6.18337359 6.19809671]
rho_best: 0.015625
qtl_best: 6.137591316570853
d_DR
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.92338758 5.92338758 5.92338758 5.92338758 5.95174946 6.24468107
 6.26293971 6.22771066 6.21461856]
rho_best: 0.0625
qtl_best: 6.309328672604761
d1_X
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.9514875 5.9514875 5.9514875 6.092349  6.2750435 6.305437  6.322032
 6.326463  6.3281045]
rho_best: 0.015625
qtl_best: 6.260542678833008
d2_X
rho:  [4. 

In [8]:
# Calculate Value
v_d1_XZ = value_fun(p_d1_XZ,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_XW = value_fun(p_d2_XW,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d_DR  = value_fun(p_d_DR, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d1_X  = value_fun(p_d1_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_X  = value_fun(p_d2_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
if proxy.cv_d1_XZ > proxy.cv_d2_XW:
    v_d4 = v_d1_XZ
else:
    v_d4 = v_d2_XW
dPESS = max(proxy.qtl_value, key = lambda k: proxy.qtl_value[k])
if dPESS == 'd1_XZ':
    v_dPESS = v_d1_XZ
elif dPESS == 'd1_X':
    v_dPESS = v_d1_X
elif dPESS == 'd2_XW':
    v_dPESS = v_d2_XW
elif dPESS == 'd2_X':
    v_dPESS = v_d2_X
else:
    v_dPESS = v_d_DR

# Table of values
pd.DataFrame.from_dict({'method':['d1_XZ','d2_XW','d4','d_DR','d1_X','d2_X', 'dPESS'],
                        'value':[v_d1_XZ,v_d2_XW,v_d4,v_d_DR,v_d1_X,v_d2_X, v_dPESS]})


,method,value
0,d1_XZ,6.215024
1,d2_XW,6.197092
2,d4,6.215024
3,d_DR,6.193553
4,d1_X,6.209450
5,d2_X,6.184956
6,dPESS,6.193553


#### Confidence interval of $V(d_3^*)$

In [9]:
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
                god=True, d_X_opt = train[["GOR_X"]])
CI = proxy.eif_CI(gamma_f='auto', n_gamma_hs=10, n_alphas=20,
            h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
            q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
            n_components=int(2*np.sqrt(samp_size)))
CI

(5.886812760679199, 6.510887764456495)

In [10]:
# V(d_3^*)
value_fun(test[["GOR_X"]].to_numpy(int).reshape(-1), test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])

6.2533986805635955

### No noisy variables in $L$

In [11]:
# Learn ITRs
rhos = np.power(2.,-np.arange(-2,7))
proxy = proxITR(A=train[["A"]], X=train[["X0","X1"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
                                learning_rate= 0.1, n_epoch=2000, opt='LBFGS', verbose=True, qtl = qtl)
d1_XZ = proxy.fit_d1_XZ_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_XZ = d1_XZ(test[["X0","X1","Z"]])

d2_XW = proxy.fit_d2_XW_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_XW = d2_XW(test[["X0","X1","W"]])

d_DR  = proxy.fit_d_DR_cv(gamma_f='auto', n_gamma_hs=10, 
        h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos, learning_rate=1e-2)
p_d_DR  = d_DR(test[["X0","X1"]])

d1_X  = proxy.fit_d1_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_X  = d1_X(test[["X0","X1"]])

d2_X  = proxy.fit_d2_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_X  = d2_X(test[["X0","X1"]])

d1_XZ
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [6.014016  6.014016  6.0154986 6.1422205 6.323061  6.354909  6.361145
 6.362258  6.362891 ]
rho_best: 0.015625
qtl_best: 6.265060615539551
d2_XW
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [6.03331282 6.03331282 6.03331282 6.03331282 6.03331282 6.03331282
 6.1148833  6.00306287 6.12897269]
rho_best: 0.015625
qtl_best: 6.059894736976906
d_DR
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [6.01829409 6.01829409 6.01829409 6.01829409 6.01829409 6.14041519
 6.31541996 6.34929852 6.35926338]
rho_best: 0.015625
qtl_best: 6.323737149243965
d1_X
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [6.014016  6.014016  6.0154986 6.139896  6.325326  6.354786  6.3619313
 6.362399  6.3647146]
rho_best: 0.015625
qtl_best: 6.265409088134765
d2_X
rho:  [4

In [12]:
# Calculate Value
v_d1_XZ = value_fun(p_d1_XZ,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_XW = value_fun(p_d2_XW,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d_DR  = value_fun(p_d_DR, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d1_X  = value_fun(p_d1_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_X  = value_fun(p_d2_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
if proxy.cv_d1_XZ > proxy.cv_d2_XW:
    v_d4 = v_d1_XZ
else:
    v_d4 = v_d2_XW
dPESS = max(proxy.qtl_value, key = lambda k: proxy.qtl_value[k])
if dPESS == 'd1_XZ':
    v_dPESS = v_d1_XZ
elif dPESS == 'd1_X':
    v_dPESS = v_d1_X
elif dPESS == 'd2_XW':
    v_dPESS = v_d2_XW
elif dPESS == 'd2_X':
    v_dPESS = v_d2_X
else:
    v_dPESS = v_d_DR

# Table of values
pd.DataFrame.from_dict({'method':['d1_XZ','d2_XW','d4','d_DR','d1_X','d2_X', 'dPESS'],
                        'value':[v_d1_XZ,v_d2_XW,v_d4,v_d_DR,v_d1_X,v_d2_X, v_dPESS]})


,method,value
0,d1_XZ,6.209243
1,d2_XW,6.134681
2,d4,6.209243
3,d_DR,6.217575
4,d1_X,6.210902
5,d2_X,6.143944
6,dPESS,6.217575


## Scenario L2

In [13]:
# Setting
linearity    = 'linear'
Xonly        = 'X'
# Selected quantile for dPESS
qtl = 0.4

In [14]:
# Generate Test Data
test = prox_data(500000, add_noise=False).gen_Y(linearity=linearity, Xonly = Xonly)
value_fun(test[["GOR"]].to_numpy(int).reshape(-1), test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])

6.22827853742395

In [15]:
# Generate Training Data
samp_size = 2000
train = prox_data(samp_size=samp_size, add_noise=True).gen_Y(linearity=linearity, Xonly = Xonly)

### With 8 noisy variables in $L$

In [16]:
# Learn ITRs
rhos = np.power(2.,-np.arange(-2,7))
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
                                learning_rate= 0.1, n_epoch=2000, opt='LBFGS', verbose=True, qtl=qtl)
d1_XZ = proxy.fit_d1_XZ_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_XZ = d1_XZ(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","Z"]])

d2_XW = proxy.fit_d2_XW_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_XW = d2_XW(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","W"]])

d_DR  = proxy.fit_d_DR_cv(gamma_f='auto', n_gamma_hs=10, 
        h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos, learning_rate=1e-2)
p_d_DR  = d_DR(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

d1_X  = proxy.fit_d1_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_X  = d1_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

d2_X  = proxy.fit_d2_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_X  = d2_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

d1_XZ
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.580619  5.7193885 6.0439234 6.097657  6.10592   6.103588  6.105073
 6.1056123 6.106444 ]
rho_best: 0.015625
qtl_best: 6.073304653167725
d2_XW
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.59061503 5.74318459 5.74965346 5.672645   5.76768782 6.10143003
 6.01196042 5.99583343 5.96308703]
rho_best: 0.125
qtl_best: 5.832251435782033
d_DR
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.71221604 5.71221604 5.71221604 5.71595492 5.87053384 6.01558491
 5.98666929 6.01197745 6.04459393]
rho_best: 0.015625
qtl_best: 5.876942617556247
d1_X
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.580619  5.7154894 6.047496  6.0999794 6.106101  6.1064196 6.106518
 6.1058025 6.1069694]
rho_best: 0.015625
qtl_best: 6.072558975219726
d2_X
rho:  [4.   

In [17]:
# Calculate Value
v_d1_XZ = value_fun(p_d1_XZ,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_XW = value_fun(p_d2_XW,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d_DR  = value_fun(p_d_DR, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d1_X  = value_fun(p_d1_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_X  = value_fun(p_d2_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
if proxy.cv_d1_XZ > proxy.cv_d2_XW:
    v_d4 = v_d1_XZ
else:
    v_d4 = v_d2_XW
dPESS = max(proxy.qtl_value, key = lambda k: proxy.qtl_value[k])
if dPESS == 'd1_XZ':
    v_dPESS = v_d1_XZ
elif dPESS == 'd1_X':
    v_dPESS = v_d1_X
elif dPESS == 'd2_XW':
    v_dPESS = v_d2_XW
elif dPESS == 'd2_X':
    v_dPESS = v_d2_X
else:
    v_dPESS = v_d_DR

# Table of values
pd.DataFrame.from_dict({'method':['d1_XZ','d2_XW','d4','d_DR','d1_X','d2_X', 'dPESS'],
                        'value':[v_d1_XZ,v_d2_XW,v_d4,v_d_DR,v_d1_X,v_d2_X, v_dPESS]})

,method,value
0,d1_XZ,6.177315
1,d2_XW,6.132798
2,d4,6.177315
3,d_DR,6.188073
4,d1_X,6.192021
5,d2_X,6.152076
6,dPESS,6.177315


#### Confidence interval of $V(d_3^*)$

In [18]:
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
                god=True, d_X_opt = train[["GOR_X"]])
CI = proxy.eif_CI(gamma_f='auto', n_gamma_hs=10, n_alphas=20,
            h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
            q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
            n_components=int(2*np.sqrt(samp_size)))
CI

(5.823806851540759, 6.425964060033808)

In [19]:
# V(d_3^*)
value_fun(test[["GOR_X"]].to_numpy(int).reshape(-1), test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])

6.22827853742395

### No noisy variables in $L$

In [20]:
# Learn ITRs
rhos = np.power(2.,-np.arange(-2,7))
proxy = proxITR(A=train[["A"]], X=train[["X0","X1"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
                                learning_rate= 0.1, n_epoch=2000, opt='LBFGS', verbose=True, qtl=qtl)
d1_XZ = proxy.fit_d1_XZ_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_XZ = d1_XZ(test[["X0","X1","Z"]])

d2_XW = proxy.fit_d2_XW_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_XW = d2_XW(test[["X0","X1","W"]])

d_DR  = proxy.fit_d_DR_cv(gamma_f='auto', n_gamma_hs=10, 
        h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos, learning_rate=1e-2)
p_d_DR  = d_DR(test[["X0","X1"]])

d1_X  = proxy.fit_d1_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_X  = d1_X(test[["X0","X1"]])

d2_X  = proxy.fit_d2_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_X  = d2_X(test[["X0","X1"]])

d1_XZ
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.6969595 5.75727   5.9549956 6.0795155 6.086185  6.0868254 6.0854387
 6.085077  6.0853887]
rho_best: 0.125
qtl_best: 5.990200138092041
d2_XW
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.73023491 5.73023491 5.73023491 5.73023491 5.73023491 5.76427354
 5.93752671 5.98221315 6.06008836]
rho_best: 0.015625
qtl_best: 5.825384740897132
d_DR
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.77547822 5.77547822 5.77547822 5.77547822 5.77547822 5.85717903
 6.14243865 6.0696809  6.08275013]
rho_best: 0.0625
qtl_best: 6.052245705655884
d1_X
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.6969595 5.75727   5.954468  6.0796204 6.083568  6.087056  6.088091
 6.0875664 6.0871954]
rho_best: 0.0625
qtl_best: 5.992678928375244
d2_X
rho:  [4.      

In [21]:
# Calculate Value
v_d1_XZ = value_fun(p_d1_XZ,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_XW = value_fun(p_d2_XW,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d_DR  = value_fun(p_d_DR, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d1_X  = value_fun(p_d1_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_X  = value_fun(p_d2_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
if proxy.cv_d1_XZ > proxy.cv_d2_XW:
    v_d4 = v_d1_XZ
else:
    v_d4 = v_d2_XW
dPESS = max(proxy.qtl_value, key = lambda k: proxy.qtl_value[k])
if dPESS == 'd1_XZ':
    v_dPESS = v_d1_XZ
elif dPESS == 'd1_X':
    v_dPESS = v_d1_X
elif dPESS == 'd2_XW':
    v_dPESS = v_d2_XW
elif dPESS == 'd2_X':
    v_dPESS = v_d2_X
else:
    v_dPESS = v_d_DR

# Table of values
pd.DataFrame.from_dict({'method':['d1_XZ','d2_XW','d4','d_DR','d1_X','d2_X', 'dPESS'],
                        'value':[v_d1_XZ,v_d2_XW,v_d4,v_d_DR,v_d1_X,v_d2_X, v_dPESS]})

,method,value
0,d1_XZ,6.198131
1,d2_XW,6.160256
2,d4,6.198131
3,d_DR,6.101156
4,d1_X,6.206344
5,d2_X,6.138676
6,dPESS,6.101156


## Scenario N1

In [22]:
# Setting
linearity    = 'nonlinear'
Xonly        = 'XW'
# Selected quantile for dPESS
qtl = 0.4

In [23]:
# Generate Test Data
test = prox_data(500000, add_noise=False).gen_Y(linearity=linearity, Xonly = Xonly)
value_fun(test[["GOR"]].to_numpy(int).reshape(-1), test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])

4.870391598434449

In [24]:
# Generate Training Data
samp_size = 2000
train = prox_data(samp_size=samp_size, add_noise=True).gen_Y(linearity=linearity, Xonly = Xonly)

### With 8 noisy variables in $L$

In [25]:
rhos = np.power(2.,-np.arange(-1,8))
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
learning_rate=0.1, n_epoch=2000, batch_size=300, opt='SGD', verbose=True, qtl=qtl)
d1_XZ = proxy.fit_d1_XZ_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_XZ = d1_XZ(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","Z"]])
d2_XW = proxy.fit_d2_XW_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_XW = d2_XW(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","W"]])
d_DR  = proxy.fit_d_DR_cv(gamma_f='auto', n_gamma_hs=10, 
        h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d_DR  = d_DR(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])
d1_X  = proxy.fit_d1_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_X  = d1_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])
d2_X  = proxy.fit_d2_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_X  = d2_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

d1_XZ
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.2986097 4.2880673 4.2853193 4.287113  4.274395  4.297773  4.2886543
 4.2935667 4.3052197]
rho_best: 0.0078125
qtl_best: 4.2843523025512695
d2_XW
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.23546225 4.22803906 4.14340169 4.21492152 4.01685271 4.03998754
 4.07499267 4.12032465 4.10362886]
rho_best: 2.0
qtl_best: 4.083964439663492
d_DR
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.17220062 4.14806023 4.15030152 4.1498565  4.18871015 4.16276233
 4.18512537 4.17072382 4.15004237]
rho_best: 0.125
qtl_best: 4.2152463282831905
d1_X
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.1093287 4.08628   4.080932  4.0735183 4.028838  4.041281  4.0226235
 4.0491076 4.018029 ]
rho_best: 2.0
qtl_best: 4.104413

In [26]:
# Calculate Value
v_d1_XZ = value_fun(p_d1_XZ,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_XW = value_fun(p_d2_XW,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d_DR  = value_fun(p_d_DR, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d1_X  = value_fun(p_d1_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_X  = value_fun(p_d2_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
if proxy.cv_d1_XZ > proxy.cv_d2_XW:
    v_d4 = v_d1_XZ
else:
    v_d4 = v_d2_XW
dPESS = max(proxy.qtl_value, key = lambda k: proxy.qtl_value[k])
if dPESS == 'd1_XZ':
    v_dPESS = v_d1_XZ
elif dPESS == 'd1_X':
    v_dPESS = v_d1_X
elif dPESS == 'd2_XW':
    v_dPESS = v_d2_XW
elif dPESS == 'd2_X':
    v_dPESS = v_d2_X
else:
    v_dPESS = v_d_DR

# Table of values
pd.DataFrame.from_dict({'method':['d1_XZ','d2_XW','d4','d_DR','d1_X','d2_X', 'dPESS'],
                        'value':[v_d1_XZ,v_d2_XW,v_d4,v_d_DR,v_d1_X,v_d2_X, v_dPESS]})

,method,value
0,d1_XZ,4.290830
1,d2_XW,4.453872
2,d4,4.290830
3,d_DR,4.218969
4,d1_X,4.216405
5,d2_X,4.210019
6,dPESS,4.290830


#### Confidence interval of $V(d_3^*)$

In [27]:
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
                god=True, d_X_opt = train[["GOR_X"]])
CI = proxy.eif_CI(gamma_f='auto', n_gamma_hs=10, n_alphas=20,
            h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
            q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
            n_components=int(2*np.sqrt(samp_size)))
CI

(4.029383450285143, 4.3666400451684195)

In [28]:
# V(d_3^*)
value_fun(test[["GOR_X"]].to_numpy(int).reshape(-1), test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])

4.250354885754258

### No noisy variables in $L$

In [29]:
rhos = np.power(2.,-np.arange(-1,8))
proxy = proxITR(A=train[["A"]], X=train[["X0","X1"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
learning_rate=0.1, n_epoch=2000, batch_size=300, opt='SGD', verbose=True, qtl=qtl)
d1_XZ = proxy.fit_d1_XZ_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_XZ = d1_XZ(test[["X0","X1","Z"]])
d2_XW = proxy.fit_d2_XW_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_XW = d2_XW(test[["X0","X1","W"]])
d_DR  = proxy.fit_d_DR_cv(gamma_f='auto', n_gamma_hs=10, 
        h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d_DR  = d_DR(test[["X0","X1"]])
d1_X  = proxy.fit_d1_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_X  = d1_X(test[["X0","X1"]])
d2_X  = proxy.fit_d2_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_X  = d2_X(test[["X0","X1"]])

d1_XZ
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.364379  4.3467884 4.366391  4.3472257 4.357066  4.347789  4.3415213
 4.3605943 4.3423157]
rho_best: 0.5
qtl_best: 4.346104717254638
d2_XW
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.2126339  4.25065518 4.2225391  4.15402895 4.20690765 4.27796796
 4.23874077 4.23369821 4.30777825]
rho_best: 0.0078125
qtl_best: 4.24772349527454
d_DR
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.17805524 4.16813309 4.17937046 4.18420187 4.16603453 4.15891751
 4.18177106 4.1890331  4.18679384]
rho_best: 0.015625
qtl_best: 4.153469099458004
d1_X
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.159342  4.151763  4.1709127 4.1756563 4.1920753 4.169508  4.1789255
 4.1923294 4.182952 ]
rho_best: 0.015625
qtl_best: 4.1

In [30]:
# Calculate Value
v_d1_XZ = value_fun(p_d1_XZ,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_XW = value_fun(p_d2_XW,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d_DR  = value_fun(p_d_DR, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d1_X  = value_fun(p_d1_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_X  = value_fun(p_d2_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
if proxy.cv_d1_XZ > proxy.cv_d2_XW:
    v_d4 = v_d1_XZ
else:
    v_d4 = v_d2_XW
dPESS = max(proxy.qtl_value, key = lambda k: proxy.qtl_value[k])
if dPESS == 'd1_XZ':
    v_dPESS = v_d1_XZ
elif dPESS == 'd1_X':
    v_dPESS = v_d1_X
elif dPESS == 'd2_XW':
    v_dPESS = v_d2_XW
elif dPESS == 'd2_X':
    v_dPESS = v_d2_X
else:
    v_dPESS = v_d_DR

# Table of values
pd.DataFrame.from_dict({'method':['d1_XZ','d2_XW','d4','d_DR','d1_X','d2_X', 'dPESS'],
                        'value':[v_d1_XZ,v_d2_XW,v_d4,v_d_DR,v_d1_X,v_d2_X, v_dPESS]})

,method,value
0,d1_XZ,4.304451
1,d2_XW,4.380044
2,d4,4.304451
3,d_DR,4.216553
4,d1_X,4.197964
5,d2_X,4.133906
6,dPESS,4.304451


## Scenario N2

In [31]:
# Setting
linearity    = 'nonlinear'
Xonly        = 'X'
# Selected quantile for dPESS
qtl = 0.4

In [33]:
# Generate Test Data
test = prox_data(500000, add_noise=False).gen_Y(linearity=linearity, Xonly = Xonly)
value_fun(test[["GOR"]].to_numpy(int).reshape(-1), test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])

4.60435930503352

In [41]:
# Generate Training Data
samp_size = 2000
train = prox_data(samp_size=samp_size, add_noise=True).gen_Y(linearity=linearity, Xonly = Xonly)

### With 8 noisy variables in $L$

In [42]:
rhos = np.power(2.,-np.arange(-1,8))
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
learning_rate=0.1, n_epoch=2000, batch_size=300, opt='SGD', verbose=True, qtl=qtl)
d1_XZ = proxy.fit_d1_XZ_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_XZ = d1_XZ(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","Z"]])
d2_XW = proxy.fit_d2_XW_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_XW = d2_XW(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","W"]])
d_DR  = proxy.fit_d_DR_cv(gamma_f='auto', n_gamma_hs=10, 
        h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d_DR  = d_DR(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])
d1_X  = proxy.fit_d1_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_X  = d1_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])
d2_X  = proxy.fit_d2_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_X  = d2_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

d1_XZ
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.5255237 4.5340533 4.5370793 4.539077  4.5376635 4.538344  4.5404077
 4.5405917 4.5388765]
rho_best: 0.015625
qtl_best: 4.526229286193848
d2_XW
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.4969357  4.47378898 4.47693317 4.39729138 4.43298152 4.43159366
 4.35032925 4.40424452 4.28193355]
rho_best: 2.0
qtl_best: 4.468713090594857
d_DR
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.52904534 4.54462684 4.58692758 4.55576565 4.56011986 4.57399574
 4.5692388  4.57850317 4.54197506]
rho_best: 0.5
qtl_best: 4.5204602507910145
d1_X
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.5265994 4.532241  4.5334644 4.5336294 4.531118  4.5330696 4.533219
 4.533044  4.5326853]
rho_best: 0.25
qtl_best: 4.5204440116

In [43]:
# Calculate Value
v_d1_XZ = value_fun(p_d1_XZ,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_XW = value_fun(p_d2_XW,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d_DR  = value_fun(p_d_DR, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d1_X  = value_fun(p_d1_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_X  = value_fun(p_d2_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
if proxy.cv_d1_XZ > proxy.cv_d2_XW:
    v_d4 = v_d1_XZ
else:
    v_d4 = v_d2_XW
dPESS = max(proxy.qtl_value, key = lambda k: proxy.qtl_value[k])
if dPESS == 'd1_XZ':
    v_dPESS = v_d1_XZ
elif dPESS == 'd1_X':
    v_dPESS = v_d1_X
elif dPESS == 'd2_XW':
    v_dPESS = v_d2_XW
elif dPESS == 'd2_X':
    v_dPESS = v_d2_X
else:
    v_dPESS = v_d_DR

# Table of values
pd.DataFrame.from_dict({'method':['d1_XZ','d2_XW','d4','d_DR','d1_X','d2_X', 'dPESS'],
                        'value':[v_d1_XZ,v_d2_XW,v_d4,v_d_DR,v_d1_X,v_d2_X, v_dPESS]})

,method,value
0,d1_XZ,4.602774
1,d2_XW,4.546749
2,d4,4.602774
3,d_DR,4.555626
4,d1_X,4.570476
5,d2_X,4.547743
6,dPESS,4.602774


#### Confidence interval of $V(d_3^*)$

In [44]:
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
                god=True, d_X_opt = train[["GOR_X"]])
CI = proxy.eif_CI(gamma_f='auto', n_gamma_hs=10, n_alphas=20,
            h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
            q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
            n_components=int(2*np.sqrt(samp_size)))
CI

(4.417750435385084, 4.804429725383926)

In [45]:
# V(d_3^*)
value_fun(test[["GOR_X"]].to_numpy(int).reshape(-1), test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])

4.60435930503352

### No noisy variables in $L$

In [39]:
rhos = np.power(2.,-np.arange(-1,8))
proxy = proxITR(A=train[["A"]], X=train[["X0","X1"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
learning_rate=0.1, n_epoch=2000, batch_size=300, opt='SGD', verbose=True, qtl=qtl)
d1_XZ = proxy.fit_d1_XZ_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_XZ = d1_XZ(test[["X0","X1","Z"]])
d2_XW = proxy.fit_d2_XW_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_XW = d2_XW(test[["X0","X1","W"]])
d_DR  = proxy.fit_d_DR_cv(gamma_f='auto', n_gamma_hs=10, 
        h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d_DR  = d_DR(test[["X0","X1"]])
d1_X  = proxy.fit_d1_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_X  = d1_X(test[["X0","X1"]])
d2_X  = proxy.fit_d2_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_X  = d2_X(test[["X0","X1"]])

d1_XZ
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.438733  4.4422164 4.446004  4.44682   4.44615   4.4492474 4.448072
 4.446369  4.449357 ]
rho_best: 0.0078125
qtl_best: 4.432885456085205
d2_XW
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.35677946 4.3601676  4.33068293 4.34075112 4.42459624 4.3706139
 4.35615776 4.44076871 4.30313906]
rho_best: 0.015625
qtl_best: 4.3489978350157354
d_DR
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.25001601 4.32931601 4.34887609 4.35236831 4.33712042 4.31970512
 4.31491592 4.32197856 4.32306092]
rho_best: 0.25
qtl_best: 4.335475639134442
d1_X
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.4376044 4.4415865 4.439994  4.4403553 4.4398413 4.4428883 4.441358
 4.4418435 4.441231 ]
rho_best: 0.0625
qtl_best: 4.422

In [40]:
# Calculate Value
v_d1_XZ = value_fun(p_d1_XZ,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_XW = value_fun(p_d2_XW,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d_DR  = value_fun(p_d_DR, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d1_X  = value_fun(p_d1_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_X  = value_fun(p_d2_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
if proxy.cv_d1_XZ > proxy.cv_d2_XW:
    v_d4 = v_d1_XZ
else:
    v_d4 = v_d2_XW
dPESS = max(proxy.qtl_value, key = lambda k: proxy.qtl_value[k])
if dPESS == 'd1_XZ':
    v_dPESS = v_d1_XZ
elif dPESS == 'd1_X':
    v_dPESS = v_d1_X
elif dPESS == 'd2_XW':
    v_dPESS = v_d2_XW
elif dPESS == 'd2_X':
    v_dPESS = v_d2_X
else:
    v_dPESS = v_d_DR

# Table of values
pd.DataFrame.from_dict({'method':['d1_XZ','d2_XW','d4','d_DR','d1_X','d2_X', 'dPESS'],
                        'value':[v_d1_XZ,v_d2_XW,v_d4,v_d_DR,v_d1_X,v_d2_X, v_dPESS]})

,method,value
0,d1_XZ,4.614229
1,d2_XW,4.488250
2,d4,4.614229
3,d_DR,4.566325
4,d1_X,4.568357
5,d2_X,4.524626
6,dPESS,4.614229
